# **Amazon DynamoDB**

**Amazon DynamoDB** è un è un **servizio cloud** interamente gestito da **Amazon Web Services (AWS)**, cui si accede attraverso **API** da qualsiasi **sistema operativo**.

Per utilizzarlo è possibile utilizzare lo [SDK](https://docs.aws.amazon.com/amazondynamodb/latest/APIReference/API_Operations.html) di **AWS**, per esempio tramite la libreria **python** <code>boto3</code>. Essendo un **servizio cloud**, **AWS** si occupa di tutte le attività di gestione del database, inclusa la configurazione, l'aggiornamento, il backup e il ripristino dei dati. Il **costo del servizio** varia in base a diversi fattori, come le **capacità di richieste di lettura e scrittura**, la **capacità di archiviazione dati** e le modalità di **backup e ripristino**. Una **descrizione dettagliata** dei vari **costi del serivizio** è disponibile al [seguente link](https://aws.amazon.com/it/dynamodb/pricing/on-demand/).

## **Vantaggi**:

Le **caratteristiche più rilevanti** del **DBMS** sono le seguenti: 
 
- **Modello chiave-valore**: DynamoDB utilizza un modello di dati chiave-valore, che offre **flessibilità** nella struttura dei dati. Si può aggiungere, rimuovere o modificare gli attributi dei dati senza dover definire uno schema rigido in anticipo;
- **Funzionalità di query flessibili**: DynamoDB offre opzioni flessibili per interrogare i dati. Si può effettuare query basate sulla chiave di partizione o utilizzare operazioni di scansione per recuperare dati in base a determinati criteri;
- **Supporto alle transazioni ACID**: Le transazioni DynamoDB forniscono agli sviluppatori **atomicità**, **consistenza**, **isolamento** e **durabilità** **(ACID)** su una o più tabelle in un unico account AWS e in un’unica regione;
- **Definzione di indici secondari (globali o locali)**: DynamoDB supporta **indici globali o locali** per consentire ricerche efficienti su attributi non chiave;
- **Scalabilità automatica**: DynamoDB **scala automaticamente** la capacità di lettura e scrittura per gestire il carico di lavoro in modo efficace. Può gestire carichi di lavoro di qualsiasi dimensione senza richiedere intervento manuale, garantendo **prestazioni elevate** e una **risposta rapida** agli accessi ai dati; 
- **Alta disponibilità e durabilità**: DynamoDB **replica** i dati in più zone di disponibilità all'interno di una regione AWS, garantendo **disponibilità** e **durabilità** elevate dei dati; 
- **Portabilità**: il servizio è facilmente accedibile da qualsiasi **sistema operativo** attraverso le sue **API**.

## **Svantaggi**

Principali svantaggi di **DynamoDB**:
- Il **modello di costo** può essere difficile da capire a volte. Ciò può rendere difficile prevedere i costi;
- **DynamoDB** non offre **JOINS** e **chiavi esterne**;
- **DynamoDB** non offre diverse **funzionalità di query** (**raggruppamenti**, **ordinamenti su attributi non chiave**, **operazioni di aggregazione**, ...).

## **Organizzazioni** che utilizzano **DynamoDB**

Alcune delle più importanti **organizzazioni** che utilizzano **DynamoDB** sono:
- **Amazon**
- **Netflix**
- **Samsung**
- **Snapchat**
- **New Yotk Times**

# **Libreria** <code>boto3</code>

Una delle **librerie** più utilizzate per accedere al servizio **DynamoDB** è <code>boto3</code>: essa fornisce un'implementazione **python** alle **API** del servizio.

La **documentazione completa** si trova al [seguente link](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

## Installazione <code>boto3</code>

In [1]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


### Librerie di comodo

<code>qtdm</code>: utilizzata per visualizzare avanzamento dei cicli

In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


<code>prettytable</code>: utilizzata per stampare risultati di query in forma tabellare

In [3]:
pip install -U prettytable

Note: you may need to restart the kernel to use updated packages.


## Importazione **librerie** e **moduli**

In [4]:
import boto3
from boto3.dynamodb.conditions import *

In [5]:
from tqdm import tqdm

In [6]:
from prettytable import PrettyTable

In [7]:
import time

## Funzione per stampa tabellare

In [8]:
def prettify_table(record_list, projection_list = None):
    out_table = PrettyTable()
    if projection_list == None:
        out_table.field_names = list(record_list[0].keys())
        out_table.add_rows([[record[field] for field in record.keys()] for record in record_list])
    else:
        out_table.field_names = projection_list
        out_table.add_rows([[record[field] for field in projection_list] for record in record_list])
    return out_table

# **Creazione** e **connessione** al DB

Per l'**analisi del database** verrà usata la versione [DynamoDB-local](https://hub.docker.com/r/amazon/dynamodb-local), che permette di testare gratuitamente il servizio tramite **docker**.

In [9]:
dynamoDB_client = boto3.client('dynamodb', 
                               endpoint_url='http://dynamoDB:8000', 
                               region_name="localhost", 
                               aws_access_key_id="false", 
                               aws_secret_access_key="false", 
                               aws_session_token="false")

In [10]:
dynamoDB_client

In [11]:
dynamoDB_resource = boto3.resource('dynamodb',
                                   endpoint_url='http://dynamoDB:8000',
                                   region_name="localhost",
                                   aws_access_key_id="false",
                                   aws_secret_access_key="false",
                                   aws_session_token="false")

In [12]:
dynamoDB_resource

dynamodb.ServiceResource()

## **Dataset** utilizzato
Raccolta di tutti i **tornei ATP** di **tennis** del 2022 in formato <code>JSON</code>

In [13]:
import json

In [14]:
with open('atp_2022.json', "r") as i_file:
    records = json.load(i_file)

Struttura di un record:

In [15]:
prettify_table([records[0]])

ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,PSW,PSL,MaxW,MaxL,AvgW,AvgL,Match_ID
1,Adelaide,Adelaide International 1,1/3/2022,ATP250,Outdoor,Hard,1st Round,3,Kwon S.W.,Nishioka Y.,53,81,1115,823,6,1,6,2,,,,,,,2,0,Completed,1.61,2.3,1.7,2.26,1.76,2.5,1.64,2.22,0


Numero di record:

In [16]:
len(records)

2632

## Popolamento del **database**

Creazione di una tabella; la **chiave primaria** è data dalla coppia <code>(ATP, Match_ID)</code>, dove:
- <code>ATP</code> è la **partition key**, ovvero la chiave usata dalla **funzione di hash**;
- <code>Match_ID</code> è **la range key**, ovvero la chiave con cui i record con la stessa **partition key** vengono ordinati

In [17]:
table = dynamoDB_resource.create_table( 
    TableName='atp_2022', 
    KeySchema=[ 
        { 
            'AttributeName': 'ATP', 
            'KeyType': 'HASH' 
        }, 
        { 
            'AttributeName': 'Match_ID', 
            'KeyType': 'RANGE' 
        } 
    ],
    AttributeDefinitions=[ 
        { 
            'AttributeName': 'ATP', 
            'AttributeType': 'N' 
        }, 
        { 
            'AttributeName': 'Match_ID', 
            'AttributeType': 'N' 
        },
    ], 
    ProvisionedThroughput={ 
        'ReadCapacityUnits': 2, 
        'WriteCapacityUnits': 2 
    } 
)

In [18]:
table

dynamodb.Table(name='atp_2022')

Creazione di un'**altra tabella** analoga alla precedente, tranne per il fatto che contiene un **indice locale secondario** sull'attributo *date*:

In [19]:
table_with_local_index = dynamoDB_resource.create_table( 
    TableName='atp_2022_local_index', 
    KeySchema=[ 
        { 
            'AttributeName': 'ATP', 
            'KeyType': 'HASH' 
        }, 
        { 
            'AttributeName': 'Match_ID', 
            'KeyType': 'RANGE' 
        } 
    ],
    LocalSecondaryIndexes=[
            {
                'IndexName': 'DateIndex',
                'KeySchema': [
                    {
                        'AttributeName': 'ATP',
                        'KeyType': 'HASH',
                    },
                    {
                        'AttributeName': 'Date',
                        'KeyType': 'RANGE',
                    }
                ],
                'Projection': {
                    'ProjectionType': 'ALL',
                }
            }
    ],
    AttributeDefinitions=[ 
        { 
            'AttributeName': 'ATP', 
            'AttributeType': 'N' 
        }, 
        { 
            'AttributeName': 'Match_ID', 
            'AttributeType': 'N' 
        },
        {
            'AttributeName': 'Date', 
            'AttributeType': 'S'
        }
    ], 
    ProvisionedThroughput={ 
        'ReadCapacityUnits': 2, 
        'WriteCapacityUnits': 2 
    } 
)

In [20]:
table_with_local_index

dynamodb.Table(name='atp_2022_local_index')

**Popolamento** della **prima tabella**:

In [21]:
cont = 0

start_time = time.time()
for i in tqdm(range(len(records))):
    result = table.put_item(Item=records[i])
    if result['ResponseMetadata']['HTTPStatusCode'] != 200:
        print(f"Fallimento nell'inserimento del record {i}: {record[i]}")
    else:
        cont += 1
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo di esecuzione: %.2f secondi" % execution_time)

100%|██████████| 2632/2632 [00:11<00:00, 229.51it/s]

Tempo di esecuzione: 11.49 secondi


Numero di **record inseriti** con **successo**:

In [22]:
assert cont == len(records), "Non sono stati inseriti tutti i record"
cont

2632

**Popolamento** della **seconda tabella**:

In [23]:
cont = 0

start_time = time.time()
for i in tqdm(range(len(records))):
    result = table_with_local_index.put_item(Item=records[i])
    if result['ResponseMetadata']['HTTPStatusCode'] != 200:
        print(f"Fallimento nell'inserimento del record {i}: {record[i]}")
    else:
        cont += 1
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo di esecuzione: %.2f secondi" % execution_time)

100%|██████████| 2632/2632 [00:10<00:00, 248.60it/s]

Tempo di esecuzione: 10.60 secondi


Numero di **record inseriti** con **successo**:

In [24]:
assert cont == len(records), "Non sono stati inseriti tutti i record"
cont

2632

Lista di **tutte le tabelle** nel database:

In [25]:
list(dynamoDB_resource.tables.all())

[dynamodb.Table(name='atp_2022'), dynamodb.Table(name='atp_2022_local_index')]

# Operazioni **CRUD**

## **Create**

### Insert one

In [26]:
start_time = time.time()
response = table.put_item(
    Item={
        'Match_ID': 27,
        'ATP': 1,
        'Location': "Adelaide",
        'Tournament': "Adelaide International 1",
        'Date': "1/3/2022",
        'Series': "ATP250",
        'Court': "Outdoor",
        'Surface': "Hard",
        'Round': "1st Qualification",
        'Best of': "3",
        'Winner': "Lorenzo Sonego",
        'Loser': "Thanasi Kokkinakis",
        'WRank': 48,
        'LRank': 76,
        'WPts': 1115,
        'LPts': 823,
        'W1': 6,
        'L1': 1,
        'W2': 6,
        'L2': 2,
        'W3': "",
        'L3': "",
        'W4': "",
        'L4': "",
        'W5': "",
        'L5': "",
        'Wsets': 2,
        'Lsets': 0,
        'Comment': "Completed",
        'B365W': "1.61",
        'B365L': "2.3",
        'PSW': "1.7",
        'PSL': "2.26",
        'MaxW': "1.76",
        'MaxL': "2.5",
        'AvgW': "1.64",
        'AvgL': "2.22"
    }
)
end_time = time.time()
execution_time = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time)
response

Tempo di esecuzione: 0.00 secondi


{'ResponseMetadata': {'RequestId': '259b9b85-508b-4ac7-8b3c-523f1053586c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 30 May 2023 16:51:57 GMT',
   'x-amzn-requestid': '259b9b85-508b-4ac7-8b3c-523f1053586c',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'content-length': '2',
   'server': 'Jetty(9.4.48.v20220622)'},
  'RetryAttempts': 0}}

### Insert many

In [27]:
start_time = time.time()
items = [
    {
         'Match_ID': 28,
        'ATP': 1,
        'Location': 'Adelaide',
        'Tournament': 'Adelaide International 1',
        'Date': '1/3/2022',
        'Series': 'ATP250',
        'Court': 'Outdoor',
        'Surface': 'Hard',
        'Round': '1st Round',
        'Best of': '3',
        'Winner': 'Kwon S.W.',
        'Loser': 'Nishioka Y.',
        'WRank': 53,
        'LRank': 81,
        'WPts': 1115,
        'LPts': 823,
        'W1': 6,
        'L1': 1,
        'W2': 6,
        'L2': 2,
        'W3': '',
        'L3': '',
        'W4': '',
        'L4': '',
        'W5': '',
        'L5': '',
        'Wsets': 2,
        'Lsets': 0,
        'Comment': 'Completed',
        'B365W': '1.61',
        'B365L': '2.3',
        'PSW': '1.7',
        'PSL': '2.26',
        'MaxW': '1.76',
        'MaxL': '2.5',
        'AvgW': '1.64',
        'AvgL': '2.22'
    },
    {
        'Match_ID': 55,
        'ATP': 2,
        'Location': 'Rome',
        'Tournament': 'Italian Open',
        'Date': '5/10/2022',
        'Series': 'ATP1000',
        'Court': 'Clay',
        'Surface': 'Outdoor',
        'Round': 'Quarterfinals',
        'Best of': '3',
        'Winner': 'Nadal R.',
        'Loser': 'Thiem D.',
        'WRank': 3,
        'LRank': 5,
        'WPts': 9800,
        'LPts': 8355,
        'W1': 7,
        'L1': 5,
        'W2': 4,
        'L2': 6,
        'W3': 6,
        'L3': 3,
        'W4': '',
        'L4': '',
        'W5': '',
        'L5': '',
        'Wsets': 2,
        'Lsets': 1,
        'Comment': 'Completed',
        'B365W': '1.45',
        'B365L': '2.75',
        'PSW': '1.5',
        'PSL': '2.7',
        'MaxW': '1.55',
        'MaxL': '2.85',
        'AvgW': '1.48',
        'AvgL': '2.63'
    }
]

with table.batch_writer() as batch:
    for item in items:
        batch.put_item(Item=item)

end_time = time.time()
execution_time = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time)   

Tempo di esecuzione: 0.01 secondi


## **Read**

**Lettura** di **tutta la tabella**:

In [28]:
start_time = time.time()
response = table.scan()
end_time = time.time()

execution_time = end_time - start_time
print(f"Tempo di esecuzione: %.2f secondi" % execution_time)

Tempo di esecuzione: 0.71 secondi


In [29]:
response.keys()

dict_keys(['Items', 'Count', 'ScannedCount', 'ResponseMetadata'])

Numero di record ritornati

In [30]:
response['Count']

2635

In [31]:
prettify_table(response['Items'][:10])

AvgL,Best of,LRank,Court,Surface,WRank,Wsets,Round,B365W,LPts,Comment,L1,L2,B365L,L3,L4,PSL,L5,MaxL,Date,Winner,WPts,Series,PSW,MaxW,Lsets,Match_ID,W1,AvgW,W2,W3,ATP,Tournament,W4,Loser,W5,Location
1.99,3,25,Outdoor,Clay,35,2,1st Round,1.8,1455,Completed,4,4,2,,,2.07,,2.1,7/18/2022,Davidovich Fokina A.,1130,ATP250,1.83,1.91,0,0,6,1.8,6,,43,European Open,,Van De Zandschulp B.,,Hamburg
6.19,3,585,Outdoor,Clay,48,2,1st Round,1.12,45,Completed,5,3,6,,,6.99,,7,7/18/2022,Molcan A.,956,ATP250,1.13,1.15,0,1,7,1.12,6,,43,European Open,,Topo M.,,Hamburg
2.26,3,75,Outdoor,Clay,62,2,1st Round,1.57,703,Completed,7,6,2.37,3,,2.33,,2.4,7/18/2022,Musetti L.,781,ATP250,1.67,1.7,1,2,6,1.62,7,6,43,European Open,,Lajovic D.,,Hamburg
6.14,3,830,Outdoor,Clay,186,2,1st Round,1.14,17,Completed,6,5,5.5,1,,6.81,,8,7/18/2022,Kovalik J.,283,ATP250,1.13,1.21,1,3,3,1.12,7,6,43,European Open,,Rehberg M.,,Hamburg
2.04,3,122,Outdoor,Clay,26,2,1st Round,1.72,443,Completed,6,3,2.1,6,,2.13,,2.15,7/18/2022,Khachanov K.,1440,ATP250,1.79,1.84,1,4,3,1.76,6,7,43,European Open,,Struff J.L.,,Hamburg
1.38,3,14,Outdoor,Clay,42,2,1st Round,3.2,2325,Completed,5,4,1.36,,,1.4,,1.42,7/19/2022,Ruusuvuori E.,1015,ATP250,3.15,3.27,0,5,7,2.99,6,,43,European Open,,Schwartzman D.,,Hamburg
1.64,3,27,Outdoor,Clay,47,2,1st Round,2.37,1412,Completed,6,5,1.57,,,1.69,,1.73,7/19/2022,Griekspoor T.,969,ATP250,2.28,2.41,0,6,7,2.23,7,,43,European Open,,Rune H.,,Hamburg
1.55,3,69,Outdoor,Clay,183,2,1st Round,2.37,741,Completed,6,6,1.57,3,,1.53,,1.62,7/19/2022,Coric B.,283,ATP250,2.65,2.76,1,7,1,2.41,7,6,43,European Open,,Djere L.,,Hamburg
4.9,3,169,Outdoor,Clay,23,2,1st Round,1.16,303,Completed,2,1,5,,,5.22,,5.34,7/19/2022,Carreno Busta P.,1510,ATP250,1.19,1.21,0,8,6,1.17,6,,43,European Open,,Nardi L.,,Hamburg
2.89,3,285,Outdoor,Clay,61,2,1st Round,1.36,170,Completed,3,6,3.2,6,,3.01,,3.26,7/19/2022,Fognini F.,789,ATP250,1.43,1.44,1,9,6,1.4,3,7,43,European Open,,Bedene A.,,Hamburg


**Lettura** con **selezione**: tutti i vincitori degli **Slam** nel 2022

In [32]:
start_time = time.time()
response = table.scan(FilterExpression = Attr('Series').eq('Grand Slam') & Attr('Round').eq('The Final'),
                      ProjectionExpression='ATP, Tournament, #loc, Winner, #d',    #Location is a reserved keyword
                      ExpressionAttributeNames = {'#loc': 'Location', '#d': 'Date'})
end_time = time.time()

execution_time = end_time - start_time
print(f"Tempo di esecuzione: %.2f secondi" % execution_time)

Tempo di esecuzione: 0.10 secondi


In [33]:
prettify_table(response['Items'])

ATP,Tournament,Date,Location,Winner
5,Australian Open,1/30/2022,Melbourne,Nadal R.
39,Wimbledon,7/10/2022,London,Djokovic N.
32,French Open,6/5/2022,Paris,Nadal R.
52,US Open,9/11/2022,New York,Alcaraz C.


## **Update**

In [34]:
start_time = time.time()

response = table.update_item(
    Key={'ATP': 1, 'Match_ID': 28 },
    ExpressionAttributeNames={
        "#attributo": "Date"
        },
        ExpressionAttributeValues={
            ':valore': '6/22/2022'
        },
        UpdateExpression="SET #attributo = :valore",
    )
end_time = time.time()

execution_time = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time)

response

Tempo di esecuzione: 0.00 secondi


{'ResponseMetadata': {'RequestId': '34e5cde7-8e08-441a-b5aa-a494964f2bd3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 30 May 2023 16:51:58 GMT',
   'x-amzn-requestid': '34e5cde7-8e08-441a-b5aa-a494964f2bd3',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'content-length': '2',
   'server': 'Jetty(9.4.48.v20220622)'},
  'RetryAttempts': 0}}

In [35]:
start_time = time.time()
response = table.update_item(
    Key={'ATP': 1, 'Match_ID': 27  },
    ExpressionAttributeNames={
        "#attributo": "WPts",
         "#attributo2": "LPts"
        },
        ExpressionAttributeValues={
            ':valore': 1100,
              ':valore2': 600
        },
        UpdateExpression="SET #attributo = :valore, #attributo2 = :valore2 ",
    )

end_time = time.time()
execution_time = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time)

response

Tempo di esecuzione: 0.01 secondi


{'ResponseMetadata': {'RequestId': '2731ec61-a0ff-4e02-9eb0-9ba503fcf0a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 30 May 2023 16:51:58 GMT',
   'x-amzn-requestid': '2731ec61-a0ff-4e02-9eb0-9ba503fcf0a0',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'content-length': '2',
   'server': 'Jetty(9.4.48.v20220622)'},
  'RetryAttempts': 0}}

## **Delete**

### Delete one

Eliminazione dell match numero  **27** del torneo con **ATP = 1**:

In [36]:
start_time = time.time()
delete_response = table.delete_item(
    Key={
        'ATP': 1,
       'Match_ID': 27
    }
)
end_time = time.time()

execution_time = end_time - start_time
print(f"Tempo di esecuzione: %.2f secondi" % execution_time)

delete_response

Tempo di esecuzione: 0.01 secondi


{'ResponseMetadata': {'RequestId': 'afe430b8-1cef-4165-b2eb-797ad1422bc4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 30 May 2023 16:51:58 GMT',
   'x-amzn-requestid': 'afe430b8-1cef-4165-b2eb-797ad1422bc4',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'content-length': '2',
   'server': 'Jetty(9.4.48.v20220622)'},
  'RetryAttempts': 0}}

### Delete many

In [37]:
start_time = time.time()
items_to_delete = [
    {'ATP': 68, 'Match_ID': 11},
    {'ATP': 51, 'Match_ID': 17},
    {'ATP': 61, 'Match_ID': 0}
]

with table.batch_writer() as batch:
    for item in items_to_delete:
        batch.delete_item(Key=item)

end_time = time.time()

execution_time = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time)


Tempo di esecuzione: 0.01 secondi


# **Comparazione** con i **DBMS** visti a lezione

**DynamoDB** si basa su un **modello chiave-valore** che offre un'elevata flessibilità nella struttura e nella organizzazione dei dati: a differenza degli modelli visti, i dati di  una tabella vengono salvati tramite una tabella di hash. E' possibile aggiungere, rimuovere o modificare gli attributi dei dati senza dover definire uno schema rigido in anticipo. Esistono due tipologie di chiavi:
- la **partition key**, la chiave utilizzata dalla funzione di hash;
- la **composite key**, formata dalla **partition key** e da una **range key**, che invece ordina i record con la stessa **partition key**;
nella creazione della tabella va specificato il tipo di chiave che si vuole utilizzare: nel caso si scelga solo la **partition key**, essa deve essere unica; altrimenti, deve essere univoca la coppia **(partition key, range query**), ovvero una **chiave primaria composita**.

Per quanto riguarda il linguaggio di interrogazione, **Amazon DynamoDB** non supporta un **query language** specifico; per eseguire **operazioni CRUD** occore utilizzare le **API** fornite da **AWS** (per esempio tramite lo [SDK](https://docs.aws.amazon.com/amazondynamodb/latest/APIReference/API_Operations.html)).

Nel caso di **boto3** con **python**, le operazioni di lettura ricordano in parte la sintassi **SQL**: è possibile eseguire **selezioni**, **proiezioni** e **limitare** i risultati in output; tuttavia, non è possibile eseguire **join**, **raggruppamenti** o **ordinamenti** (se non sulla **range key**, effettuato di default ad ogni interrogazione). 

E' inoltre possibile definire due tipologie di **indici secondari**
- **indici secondari locali**:  utilizzano la stessa **partition key** dell'indice primario ma permettono di utilizzare una diversa **range key**; di conseguenza essi possono essere definiti solo su tabelle con una **chiave primaria composita**;
- **indici secondari globali**: non è necessario che condividano la stessa **partition key** dell'indice primario.

La **documentazione ufficiale** è consultabile al seguente link: https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Introduction.html

# Esempi di **query**

1) **Ricerca** per **chiave**: **tutte** le **partite** dell'ATP 29

**Senza indice primario** (scansione di tutto il db):

In [38]:
start_time = time.time()
response_no_index = table.scan(FilterExpression = Attr('ATP').eq(29),
                      ProjectionExpression = 'ATP, Series, Surface, Match_ID, Round, Tournament, #loc, Winner, Loser, #d',    #Location is a reserved keyword
                      ExpressionAttributeNames = {'#loc': 'Location','#d': 'Date'})
end_time = time.time()
execution_time_1_noindex  = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time_1_noindex)

Tempo di esecuzione: 0.15 secondi


**Tramite indice primario**:

In [39]:
start_time = time.time()
response_index = table.query(KeyConditionExpression = Key('ATP').eq(29),
                      ProjectionExpression = 'ATP, Series, Surface, Match_ID, Round, Tournament, #loc, Winner, Loser, #d',    #Location is a reserved keyword
                      ExpressionAttributeNames = {'#loc': 'Location','#d': 'Date'},
                      ScanIndexForward = False)
end_time = time.time()
execution_time_1_index  = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time_1_index)

Tempo di esecuzione: 0.02 secondi


In [40]:
prettify_table(response_index['Items'][:20], ['ATP', 'Tournament', 'Location', 'Series', 'Round', 'Date', 'Match_ID', 'Surface', 'Winner', 'Loser'])

ATP,Tournament,Location,Series,Round,Date,Match_ID,Surface,Winner,Loser
29,Internazionali BNL d'Italia,Rome,Masters 1000,The Final,5/15/2022,54,Clay,Djokovic N.,Tsitsipas S.
29,Internazionali BNL d'Italia,Rome,Masters 1000,Semifinals,5/14/2022,53,Clay,Djokovic N.,Ruud C.
29,Internazionali BNL d'Italia,Rome,Masters 1000,Semifinals,5/14/2022,52,Clay,Tsitsipas S.,Zverev A.
29,Internazionali BNL d'Italia,Rome,Masters 1000,Quarterfinals,5/13/2022,51,Clay,Djokovic N.,Auger-Aliassime F.
29,Internazionali BNL d'Italia,Rome,Masters 1000,Quarterfinals,5/13/2022,50,Clay,Ruud C.,Shapovalov D.
29,Internazionali BNL d'Italia,Rome,Masters 1000,Quarterfinals,5/13/2022,49,Clay,Tsitsipas S.,Sinner J.
29,Internazionali BNL d'Italia,Rome,Masters 1000,Quarterfinals,5/13/2022,48,Clay,Zverev A.,Garin C.
29,Internazionali BNL d'Italia,Rome,Masters 1000,3rd Round,5/12/2022,47,Clay,Shapovalov D.,Nadal R.
29,Internazionali BNL d'Italia,Rome,Masters 1000,3rd Round,5/12/2022,46,Clay,Ruud C.,Brooksby J.
29,Internazionali BNL d'Italia,Rome,Masters 1000,3rd Round,5/12/2022,45,Clay,Djokovic N.,Wawrinka S.


2) **Range query**: tutte le partite dell'**ATP 23** tra il **14 Aprile** e il **19 Aprile** vinte da **Stefanos Tsitsipas**:

Senza **LSI**:

In [41]:
start_time = time.time()
response_no_index = table.scan(FilterExpression = Attr('ATP').eq(23) & Attr('Date').between('4/14/2022', '4/19/2022')
                               & Attr('Winner').eq('Tsitsipas S.'),
                               ProjectionExpression='ATP, Tournament, #loc, Round, #d, Winner, Match_ID, Loser',    #Location, Date are reserved keywords
                               ExpressionAttributeNames = {'#loc': 'Location', '#d': 'Date'})
end_time = time.time()
execution_time_2_noindex  = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time_2_noindex)

Tempo di esecuzione: 0.35 secondi


Con **LSI** su *Date*:

In [42]:
start_time = time.time()
response_index = table_with_local_index.query(IndexName='DateIndex',
                                              KeyConditionExpression = Key('ATP').eq(23) & Key('Date').between('4/14/2022', '4/19/2022'),
                                              FilterExpression = Attr('Winner').eq('Tsitsipas S.'),
                                              ProjectionExpression='ATP, Tournament, #loc, Round, #d, Winner, Match_ID, Loser',    #Location, Date are reserved keywords
                                              ExpressionAttributeNames = {'#loc': 'Location', '#d': 'Date'})
end_time = time.time()
execution_time_2_index  = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time_2_index)

Tempo di esecuzione: 0.03 secondi


In [43]:
assert response_no_index['Items'] == response_index['Items'], "Differents queries results"
prettify_table(response_index['Items'])

Match_ID,Round,ATP,Tournament,Date,Loser,Location,Winner
44,3rd Round,23,Monte Carlo Masters,4/14/2022,Djere L.,Monte Carlo,Tsitsipas S.
51,Quarterfinals,23,Monte Carlo Masters,4/15/2022,Schwartzman D.,Monte Carlo,Tsitsipas S.
53,Semifinals,23,Monte Carlo Masters,4/16/2022,Zverev A.,Monte Carlo,Tsitsipas S.
54,The Final,23,Monte Carlo Masters,4/17/2022,Davidovich Fokina A.,Monte Carlo,Tsitsipas S.


3) Tutti i **giocatori** che hanno battuto **Jannick Sinner** in **meno di 4 set** nel 2022 su **terra rossa**:

In [44]:
start_time = time.time()
response = table.scan(FilterExpression = Attr('Loser').eq('Sinner J.') & 
                      Attr('W4').eq('') & Attr('L4').eq('') & Attr('Surface').eq('Clay'),
                      ProjectionExpression='Tournament, #loc, Surface, Round, #d, Winner, Loser, W1, L1, W2, L2, W3, L3',    #Location, Date are reserved keywords
                      ExpressionAttributeNames = {'#d': 'Date', '#loc': 'Location'})
end_time = time.time()
execution_time_3  = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time_3)

Tempo di esecuzione: 0.33 secondi


In [45]:
prettify_table(response['Items'], ['Tournament', 'Location', 'Round', 'Date', 'Winner', 'Loser', 'Surface', 'W1', 'W2', 'W3', 'L1', 'L2', 'L3'])

Tournament,Location,Round,Date,Winner,Loser,Surface,W1,W2,W3,L1,L2,L3
Monte Carlo Masters,Monte Carlo,Quarterfinals,4/15/2022,Zverev A.,Sinner J.,Clay,5,6,7,7,3,6
Mutua Madrid Open,Madrid,3rd Round,5/5/2022,Auger-Aliassime F.,Sinner J.,Clay,6,6,,1,2,
Internazionali BNL d'Italia,Rome,Quarterfinals,5/13/2022,Tsitsipas S.,Sinner J.,Clay,7,6,,6,2,
French Open,Paris,4th Round,5/30/2022,Rublev A.,Sinner J.,Clay,1,6,2,6,4,0


## Definizione di un **Global Secondary Index (GSI)**

Definizione di un **GSI** basato sulla **compostite key** <code>(Tournament, Match_ID)</code>:

In [46]:
 resp = dynamoDB_client.update_table(
        TableName="atp_2022",
        GlobalSecondaryIndexUpdates=[
            {
                "Create": {
                    "IndexName": "TournamentIndex",
                    "KeySchema": [
                        {
                            "AttributeName": "Tournament",
                            "KeyType": "HASH"
                        },
                        {
                            "AttributeName": "Match_ID",
                            "KeyType": "RANGE"
                        }
                    ],
                    "Projection": {
                        "ProjectionType": "ALL"
                    },
                    "ProvisionedThroughput": {
                        "ReadCapacityUnits": 1,
                        "WriteCapacityUnits": 1,
                    }
                }
            }
        ],
        AttributeDefinitions=[
            {
                "AttributeName": "Tournament",
                "AttributeType": "S"
            },
            {
                "AttributeName": "Match_ID",
                "AttributeType": "N"
            }
        ],
    )

In [47]:
resp['TableDescription']['GlobalSecondaryIndexes']

[{'IndexName': 'TournamentIndex',
  'KeySchema': [{'AttributeName': 'Tournament', 'KeyType': 'HASH'},
   {'AttributeName': 'Match_ID', 'KeyType': 'RANGE'}],
  'Projection': {'ProjectionType': 'ALL'},
  'IndexStatus': 'CREATING',
  'Backfilling': False,
  'ProvisionedThroughput': {'ReadCapacityUnits': 1, 'WriteCapacityUnits': 1},
  'IndexArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/atp_2022/index/TournamentIndex'}]

4) Le ultime **15 partite** del torneo di **Wimbledon**:

Senza **GSI** **(Query innestata)**:

In [48]:
start_time = time.time()
response_no_index = table.query(KeyConditionExpression = Key('ATP').eq(
    table.scan(FilterExpression = Attr('Tournament').eq('Wimbledon'),
               ProjectionExpression='ATP'
              )['Items'][0]['ATP']),
                                ProjectionExpression='Tournament, #loc, Round, #d, Winner, Match_ID, Loser',    #Location, Date are reserved keywords
                                ExpressionAttributeNames = {'#loc': 'Location', '#d': 'Date'},
                                ScanIndexForward = False,
                                Limit = 15
                               )
end_time = time.time()
execution_time_4_noindex  = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time_4_noindex)

Tempo di esecuzione: 0.83 secondi


Con **GSI** su *Tournament*:

In [49]:
start_time = time.time()
response_index = table.query(IndexName = 'TournamentIndex',
                             KeyConditionExpression = Key('Tournament').eq('Wimbledon'),
                             ProjectionExpression='Tournament, #loc, Round, #d, Winner, Match_ID, Loser',    #Location, Date are reserved keywords
                             ExpressionAttributeNames = {'#loc': 'Location', '#d': 'Date'},
                             ScanIndexForward = False,
                             Limit = 15)
end_time = time.time()
execution_time_4_index  = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time_4_index)

Tempo di esecuzione: 0.01 secondi


In [50]:
assert response_index['Items'] == response_no_index['Items'], "Differents queries results"
prettify_table(response_index['Items'], ['Tournament', 'Location', 'Round', 'Date', 'Match_ID', 'Winner', 'Loser'])

Tournament,Location,Round,Date,Match_ID,Winner,Loser
Wimbledon,London,The Final,7/10/2022,126,Djokovic N.,Kyrgios N.
Wimbledon,London,Semifinals,7/8/2022,125,Kyrgios N.,Nadal R.
Wimbledon,London,Semifinals,7/8/2022,124,Djokovic N.,Norrie C.
Wimbledon,London,Quarterfinals,7/6/2022,123,Kyrgios N.,Garin C.
Wimbledon,London,Quarterfinals,7/6/2022,122,Nadal R.,Fritz T.
Wimbledon,London,Quarterfinals,7/5/2022,121,Norrie C.,Goffin D.
Wimbledon,London,Quarterfinals,7/5/2022,120,Djokovic N.,Sinner J.
Wimbledon,London,4th Round,7/4/2022,119,Nadal R.,Van De Zandschulp B.
Wimbledon,London,4th Round,7/4/2022,118,Fritz T.,Kubler J.
Wimbledon,London,4th Round,7/4/2022,117,Kyrgios N.,Nakashima B.


5) Tutti i **giocatori** che hanno battuto un **giocatore** sopra in classifica al **vincitore** del torneo **Australian Open**:

Senza **GSI**:

In [51]:
start_time = time.time()
response_no_index = table.scan(FilterExpression = Attr('WRank').gt(10) &
                            Attr('LRank').lt(
                                table.scan(
                                        FilterExpression = Attr('Round').eq('The Final') &
                                        Attr('Tournament').eq('Australian Open'),
                                        ProjectionExpression = 'WRank'
                            )['Items'][0]['WRank']),
                            ProjectionExpression='Tournament, #loc, Round, #d, Winner, Loser, WRank, LRank, WPts, LPts',    #Location, Date are reserved keywords
                            ExpressionAttributeNames = {'#loc': 'Location', '#d': 'Date'}
                            )
end_time = time.time()
execution_time_5_noindex  = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time_5_noindex)

Tempo di esecuzione: 0.44 secondi


Con **GSI** nella query innestata:

In [52]:
start_time = time.time()
response_index = table.scan(FilterExpression = Attr('WRank').gt(10) &
                            Attr('LRank').lt(
                                table.query(
                                            KeyConditionExpression = Key('Tournament').eq('Australian Open'),
                                            IndexName = "TournamentIndex",
                                            FilterExpression = Attr('Round').eq('The Final'),
                                            ProjectionExpression = 'WRank'
                                )['Items'][0]['WRank']),
                            ProjectionExpression='Tournament, #loc, Round, #d, Winner, Loser, WRank, LRank, WPts, LPts',    #Location, Date are reserved keywords
                            ExpressionAttributeNames = {'#loc': 'Location', '#d': 'Date'}
                            )
end_time = time.time()
execution_time_5_index  = end_time - start_time
print("Tempo di esecuzione: %.2f secondi" % execution_time_5_index)

Tempo di esecuzione: 0.22 secondi


In [53]:
assert response_index['Items'] == response_no_index['Items'], "Differents queries results"
prettify_table(response_no_index['Items'], ['Winner', 'WPts', 'WRank', 'Loser', 'LPts', 'LRank', 'Tournament', 'Location', 'Date', 'Round'])

Winner,WPts,WRank,Loser,LPts,LRank,Tournament,Location,Date,Round
Munar J.,814,58,Ruud C.,5645,3,Rakuten Japan Open Tennis Championships,Tokyo,10/4/2022,1st Round
Davidovich Fokina A.,1095,46,Djokovic N.,8420,1,Monte Carlo Masters,Monte Carlo,4/12/2022,2nd Round
Goffin D.,770,66,Alcaraz C.,6740,1,Astana Open,Nur-Sultan,10/4/2022,1st Round
Gasquet R.,808,75,Medvedev D.,7980,2,Geneva Open,Geneva,5/17/2022,2nd Round
Wawrinka S.,175,284,Medvedev D.,5065,4,Open de Moselle,Metz,9/22/2022,2nd Round
De Minaur A.,1710,25,Medvedev D.,5655,3,BNP Paribas Masters,Paris,11/2/2022,2nd Round
Paul T.,1330,31,Nadal R.,5810,2,BNP Paribas Masters,Paris,11/2/2022,2nd Round
Musetti L.,1746,23,Ruud C.,5510,4,BNP Paribas Masters,Paris,11/3/2022,3rd Round
Rune H.,1991,18,Alcaraz C.,6730,1,BNP Paribas Masters,Paris,11/4/2022,Quarterfinals
Shapovalov D.,2593,14,Zverev A.,7970,3,Australian Open,Melbourne,1/23/2022,4th Round


# **Analisi** delle **tempistiche delle query**

In [54]:
APPROX = 2

In [55]:
time_analysis = []
time_analysis.append({'Number': 1, 
                      'Query type': 'Key-value search', 
                      'Index type': 'Primary Key',
                      'Time without index (s)': round(execution_time_1_noindex, APPROX), 
                      'Time with index (s)': round(execution_time_1_index, APPROX), 
                      'Time saved (s)': round(execution_time_1_noindex - execution_time_1_index, APPROX),
                      'Time saved (%)': round((execution_time_1_noindex - execution_time_1_index) / execution_time_1_noindex, APPROX)
                     })

time_analysis.append({'Number': 2, 
                      'Query type': 'Range query', 
                      'Index type': 'LSI',
                      'Time without index (s)': round(execution_time_2_noindex, APPROX), 
                      'Time with index (s)': round(execution_time_2_index, APPROX), 
                      'Time saved (s)': round(execution_time_2_noindex - execution_time_2_index, APPROX),
                      'Time saved (%)': round((execution_time_2_noindex - execution_time_2_index) / execution_time_2_noindex, APPROX)
                     })

time_analysis.append({'Number': 3, 
                      'Query type': 'Multiple selections', 
                      'Index type': '',
                      'Time without index (s)': round(execution_time_3, APPROX), 
                      'Time with index (s)': '',
                      'Time saved (s)': '',
                      'Time saved (%)': ''
                     })

time_analysis.append({'Number': 4, 
                      'Query type': 'Desc ordered selection', 
                      'Index type': 'GSI',
                      'Time without index (s)': round(execution_time_4_noindex, APPROX), 
                      'Time with index (s)': round(execution_time_4_index, APPROX), 
                      'Time saved (s)': round(execution_time_4_noindex - execution_time_4_index, APPROX),
                      'Time saved (%)': round((execution_time_4_noindex - execution_time_4_index) / execution_time_4_noindex, APPROX)
                     })

time_analysis.append({'Number': 5, 
                      'Query type': 'Nested query', 
                      'Index type': 'GSI on nested query',
                      'Time without index (s)': round(execution_time_5_noindex, APPROX), 
                      'Time with index (s)': round(execution_time_5_index, APPROX), 
                      'Time saved (s)': round(execution_time_5_noindex - execution_time_5_index, APPROX),
                      'Time saved (%)': round((execution_time_5_noindex - execution_time_5_index) / execution_time_5_noindex, APPROX)
                     })

In [56]:
prettify_table(time_analysis)

Number,Query type,Index type,Time without index (s),Time with index (s),Time saved (s),Time saved (%)
1,Key-value search,Primary Key,0.15,0.02,0.12,0.85
2,Range query,LSI,0.35,0.03,0.32,0.91
3,Multiple selections,,0.33,,,
4,Desc ordered selection,GSI,0.83,0.01,0.83,0.99
5,Nested query,GSI on nested query,0.44,0.22,0.22,0.5


## **Delete** di una tabella

In [57]:
dynamoDB_client.delete_table(TableName='atp_2022')

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'Match_ID',
    'AttributeType': 'N'},
   {'AttributeName': 'ATP', 'AttributeType': 'N'},
   {'AttributeName': 'Tournament', 'AttributeType': 'S'}],
  'TableName': 'atp_2022',
  'KeySchema': [{'AttributeName': 'ATP', 'KeyType': 'HASH'},
   {'AttributeName': 'Match_ID', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2023, 5, 30, 16, 51, 34, 967000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 2,
   'WriteCapacityUnits': 2},
  'TableSizeBytes': 820115,
  'ItemCount': 2631,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/atp_2022',
  'GlobalSecondaryIndexes': [{'IndexName': 'TournamentIndex',
    'KeySchema': [{'AttributeName': 'Tournament', 'KeyType': 'HAS

In [58]:
dynamoDB_client.delete_table(TableName='atp_2022_local_index')

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'ATP',
    'AttributeType': 'N'},
   {'AttributeName': 'Match_ID', 'AttributeType': 'N'},
   {'AttributeName': 'Date', 'AttributeType': 'S'}],
  'TableName': 'atp_2022_local_index',
  'KeySchema': [{'AttributeName': 'ATP', 'KeyType': 'HASH'},
   {'AttributeName': 'Match_ID', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2023, 5, 30, 16, 51, 35, 35000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 2,
   'WriteCapacityUnits': 2},
  'TableSizeBytes': 820451,
  'ItemCount': 2632,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/atp_2022_local_index',
  'LocalSecondaryIndexes': [{'IndexName': 'DateIndex',
    'KeySchema': [{'AttributeName': 'ATP', 'KeyType': '

# Contributors
- **Volpato Mattia 866316**
- **Bazzocchi Tommaso 869095**